# Metaklasy

Wszystko jest obiektem - tak jak instancje mają swoje klasy/typy które określają ich zachowanie, tak klasy mają swoje klasy/typy, które określają ich zachowanie. Są to właśnie metaklasy. Podstawową metaklasą jest type (tak, to type, które mówi nam jakiego typu są obiekty); type jest metaklasą dla samego siebie.

Metaklasy to funkcje albo klasy, które powinny zwrócić nową klasę.

Cały flow tworzenia klasy jest następujący:
1. określana jest metaklasa
2. przygotowywany jest namespace dla klasy (metoda \_\_prepare\_\_)
3. wykonywane jest ciało klasy
4. tworzony jest obiekt klasy

In [ ]:
from __future__ import print_function
print(type(2))
print(type(int))
print(type(type))

In [ ]:
# alternatywna do class metoda tworzenia klas - type z trzema argumetnami:
# type(NazwaKlasy, KrotkaKlasBazowych, SłownikAtrybutów)

Kotek = type(
    'Kotek',  # klasa będzie miała nazwę Kotek - nazwa może być inna niż zmienna, do której jest przypisywana
    (), # nie ma zadnych dodatkowych klas bazowych, będzie tylko object
    {'co_lubi': 'mleko', 'f': lambda self: self.co_lubi})  # atrybut klasy *co_lubi* i metoda *f*

kotek  = Kotek()
print('kotek.co_lubi:', kotek.co_lubi)
print('vars(kotek):', vars(kotek))  # instancja nie ma dodatkowych atrybutów
print('vars(Kotek):', vars(Kotek))  # co_lubi jest atrybutem klasy
print('Kotek.mro():', Kotek.mro())
kotek.f()

In [ ]:
# metaklasa w formie funkcji przyjmuje 3 argumenty -
# nazwę klasy, listę klas bazowych i namespace - atrybuty i metody
def metaklasa(name, bs, ns):
    print('metaklasa pozdrawia')
    print('name: ', name)
    print('bs', bs)  # bs - Bases
    print('ns', ns)  # ns - Namespace
    return type(name, bs, ns)

# jeżeli metaklasa jest funkcją, trzeba ją ręcznie podpiąć do każdej klasy, w której chcemy ją mieć

In [ ]:
# Python 2

# metaklasę podpina się do atrybutu __metaclass__
class A(object):
    __metaclass__ = metaklasa

class B(A):
    __metaclass__ = metaklasa  # nie jest konieczne, żeby klasa bazowa miała podpiętą metaklasę
    x = 23
    def f(self):
        print(self)

In [ ]:
# Python 3

# metaklasę podaje się przy definicji klasy, po klasach bazowych - jako keyword
class A(metaclass=metaklasa):
    pass

print('>>')
class B(A, metaclass=metaklasa):
    x = 23
    def f(self):
        print(self)

generalnie metaklasa wcale nie musi wywoływać type i tworzyć obiektu określonej klasy,
może na przykałd zwracać int

In [ ]:
# Python 2
def metaklasa(*a):
    return 2

class A:
    __metaclass__ = metaklasa
    

print(A)  # A to jest liczba 2 a nie <class '__main__.A'>

In [ ]:
# Python 3
def metaklasa(*a):
    return 2

class A(metaclass=metaklasa):
    pass
    
print(A)  # A to jest liczba 2 a nie <class '__main__.A'>

In [ ]:
# w praktyce, metaklasy to klasy, które dziedziczą po type
# (oczywiscie nie muszą, ale wtedy trzeba samemu zatroszyc się o odpowiednie tworzenie
# obiektów)

class Meta(type):
    def __new__(cls, name, bs, ns):
        # __new__ to classmethod  - cls jest metaklasą Meta
        # przechwytywanie procesu tworzenia klasy - tego co będzie przypisane do zmiennej
        # o znawie name
        print('tworzenie obiektu')
        # można wpływac na atrybuty, mro, i nazwę klasy, ale teraz nie zrobimy nic
        return super(Meta, cls).__new__(cls, name, bs, ns)
    
    def __init__(self, name , bs, ns):
        # na tym etapie self to juz utworzona klasa
        # inicjalizacja klasy - nazwa jest już nadana, to za późno, żeby to zmienić,
        # tak samo klasy bazowe i atrybuty
        print("inicjalizacja")
        print( name, bs, ns)
        name = name.upper()
        bs = ()
        ns = {'x': 24}
        return super(Meta, self).__init__(name, bs, ns)
    
    def __call__(self, *a, **kw):
        # tak jak a() wywoływało metodę __call__ z klasy na rzecz tego obiektu,
        # tak A() wywołuje __call__ metaklasy, na rzecz tej klasy
        # jeżeli klasa nie jest wywoływalna (nie ma __call__ w metaklasie to nie można
        # stworzyć instancji klasy)
        print('Meta.__call__')
        return super(Meta, self).__call__(*a, **kw)

In [ ]:
# Python 2
class A(object):
    __metaclass__ = Meta

In [ ]:
# Python 2
A()

In [ ]:
# Python 2
class Aasdf(A):  # jeżeli klasa bazowa nie ma zdefiniowanej metaklasy,
    x = 23       # to uzywana jest ta z klasy bazowej - metaklasy w formie klas  są dziedziczone

In [ ]:
# Python 2
print('Aasdf:', Aasdf)
print('Aasdf.__name__:', Aasdf.__name__)  # __init__ nie wpływa na nazwę klasy
print('Aasdf.mro():', Aasdf.mro())  # __init__ nie wpływa na mro
print('dir(Aasdf):', dir(Aasdf))  # __init__ nie wpływa na listę atrybutów
print('Aasdf.x:', Aasdf.x)  # __init__ nie wpływa na wartosći atrybutów

In [ ]:
# Python 2
a = Aasdf()

In [ ]:
# Python 2
# alternatywnie można towrzyć klasy wywołujac metaklasę - dokładnie tak jak tworząc instancję klasy wywoływać klase -
# bo w rzeczywistości klasy są instancjami metaklas
Klaska = Meta('Klaska', (), {})  # wywołanie takie jak type(..., ..., ...)
print(Klaska)

In [ ]:
# Python 3
class A(metaclass=Meta):
    pass

In [ ]:
# Python 3
A()  # tworzenie instancji -> Meta.__call__

In [ ]:
# Python 3
class Aasdf(A):  # jeżeli klasa bazowa nie ma zdefiniowanej metaklasy,
    x = 23       # to uzywana jest ta z klasy bazowej - metaklasy w formie klas są dziedziczone

In [ ]:
# Python 3
print('Aasdf:', Aasdf)
print('Aasdf.__name__:', Aasdf.__name__)  # __init__ nie wpływa na nazwę klasy
print('Aasdf.mro():', Aasdf.mro())  # __init__ nie wpływa na mro
print('dir(Aasdf):', dir(Aasdf))  # __init__ nie wpływa na listę atrybutów
print('Aasdf.x:', Aasdf.x)  # __init__ nie wpływa na wartosći atrybutów

In [ ]:
# Python 3
a = Aasdf()

In [ ]:
# Python 3
# alternatywnie można towrzyć klasy wywołujac metaklasę - dokładnie tak jak tworząc instancję klasy wywoływać klase -
# bo w rzeczywistości klasy są instancjami metaklas
Klaska = Meta('Klaska', (), {})  # wywołanie takie jak type(..., ..., ...)
print(Klaska)

Przykład

In [ ]:
class BSCMeta(type):
    def __getitem__(self, item):
        print(self)
        if isinstance(item, slice):
            return [self(i) for i in range(item.start, item.stop, item.step or 1)]
        return self(item)

In [ ]:
# Python 2
class BSC:
    __metaclass__ = BSCMeta
    def __init__(self, id):
        self.id = id      


In [ ]:
#Python 3
class BSC(metaclass=BSCMeta):
    def __init__(self, id):
        self.id = id      


Skoro już wiadomo, że stworzenie instancji klasy, to wywołanie klasy, to można zastosować dekorator partial, również do klas

In [ ]:
import sys

from functools import partial
from io import StringIO


class Logger(object):
    def __init__(self, file, config=None):
        print('file, config:', file, config)
        self.file = file
        self.config = config
        
        
stdout_logger = partial(Logger, sys.stdout)
file_logger = partial(Logger, StringIO())
ls1 = stdout_logger(1)  # tylko config
ls2 = stdout_logger(2)
lf1 = file_logger(1)  # Uwaga, bo instancja StringIO jest ta sama dla obydwóch loggerów lf1 i lf2
lf2 = file_logger(2)

## Python 3 - keywordy w klasach

w Pythonie 3, podaczas definiowania klasy, można przekazywać dodatkowe paramerty do metaklasy

In [ ]:
# Python 3
def meta(name, bases, namespace, keyword=None):  # argument keywords
    print(name, bases, namespace, keyword, sep='|')
    return type(name, bases, namespace)

class A(metaclass=meta, keyword=3):
    pass

In [ ]:
class Meta(type):
    def __new__(mcls, name, bases, namespace, keyword=None):  # do __new__ i __init__ nalezy dodać kolejny argument
        print(name, bases, namespace, keyword, sep='|')
        return super().__new__(mcls, name, bases, namespace)
    
class A(metaclass=Meta, keyword=3):
    pass    

## Python 3.6 - __init_subclass__

W klasie można zdefiniować metodę statyczną \_\_init\_subclass\_\_.

Do \_\_init\_subclass\_\_ nie jest przekazywana metaklasy.

In [ ]:
class A:
    def __init_subclass__(cls, *args, **kwargs):
        print('__init_subclass__')
        print(cls, args, kwargs)
    
    
# cls to klasa B, args i kwargs są puste
class B(A):
    pass

# cls to C, args są puste, keywords zawiera klucz keyword z wartością 123
class C(A, keyword=123):
    pass

### Python 3 - przygotowywanie namespacu

w Pythonie 3 można zaimplementować metodę \_\_prepare\_\_ w metaklasie, która służy do przygotowania przestrzeni klasy, która potem bedzie uzupełniana podczas tworzenia klasy.
\_\_prepare\_\_ jest wywoływana przed wywołaniem \_\_new\_\_

In [ ]:
class Meta(type):
    @classmethod  # to musi być metaklasa a nie klasa - jeszcze klasa nie istnieje
    def __prepare__(mcls, name, bases, **kwargs):  # nie ma tutaj argumentu namespace - namespace będzie tworzona
        return {'a': 33}
    
class A(metaclass=Meta):
    pass

print(dir(A))  # jest atrybut 'a'

## klasy abstrakcyjne
moduł abc (Abstract Base Classes) zawiera metaklasę ABCMeta, która jest uzywana do tworzenia 
m.in metod abstrakcyjnych, ale równiez kilku innyc hciekawych rzeczy

In [ ]:
# Python 2
# można dodac 'virtualne klasy bazowe' przy pomocy funkcji register

import abc


class Sekwencja:
    __metaclass__ = abc.ABCMeta

In [ ]:
# Python 3
# można dodac 'virtualne klasy bazowe' przy pomocy funkcji register

import abc


class Sekwencja(metaclass=abc.ABCMeta):
    pass

In [ ]:
# Python2/3
    
Sekwencja.register(tuple)  # krotka będzie widziana jako podklasa Sekwencji


print(tuple.mro())  # Sekwencji nie ma w mro


print(issubclass(tuple, Sekwencja))  # ale i tak pokazuje że krotka jest podklasą
print(isinstance((), Sekwencja))

print(issubclass(list, Sekwencja))  # lista nie jest tak postrzegana
print(isinstance([], Sekwencja))

print(40 * '=')

class Sekwencja2(Sekwencja):
    pass

print(issubclass(tuple, Sekwencja2))  # funkcja register nie działa dla klas dizedziczącyk po
print(isinstance((), Sekwencja2))     # wirtualnej klasie bazowej

In [ ]:
# Python2
# alternatywnie można zaimplementować metodę __subclasshook__ jako classmethod

import abc


class Sekwencja :
    __metaclass__ = abc.ABCMeta
    
    @classmethod
    def __subclasshook__(cls, subclass):
        if subclass in [tuple, list]:  # krotki i listy są uważane za podklasy
            return True
        return False

In [ ]:
# Python 3
# alternatywnie można zaimplementować metodę __subclasshook__ jako classmethod

import abc


class Sekwencja(metaclass=abc.ABCMeta):
    pass
    
    @classmethod
    def __subclasshook__(cls, subclass):
        if subclass in [tuple, list]:  # krotki i listy są uważane za podklasy
            return True
        return False

In [ ]:
# Python 2/3
print(tuple.mro())  # Sekwencji nie ma w mro


print(issubclass(tuple, Sekwencja))  # ale i tak pokazuje że krotka jest podklasą
print(isinstance((), Sekwencja))


print(40 * '=')

class Sekwencja2(Sekwencja):
    pass

print(issubclass(tuple, Sekwencja2))  # działa dla klasy pochodnej - register nie działało
print(isinstance((), Sekwencja2))


# metodę __subclasshook__ i funkcję register można łączyć - wtedy "lista podklas"
# to suma zbiorów klas zarejestrowanych i tych określonych przez metodę __subclasshook__

### metody i property abstrakcyjne

In [ ]:
# tak drzewiej bywało:

class Abstrakcyjna(object):
    # dużo metod
    def metoda_abstrakcyjna(self):
        raise NotImplementedError() # generalnie zadziała
        
        
class NieAbstrakcyjna(Abstrakcyjna):
    # programista naspisuje inne metody - tylko zapomniał o tej o nazwie metoda_abstrakcyjna
    pass
        
    
a = NieAbstrakcyjna()  # przejdzie, a nie powinno, bo programista nie nadpisał metody abstrakcyjnej

# w zupełnie innym pliku 2000 linii kodu dalej
a.metoda_abstrakcyjna()  # wykrzaczy się - dużo debugowania
        


In [ ]:
# Python 2
# metody abstrakcyjne muszą być nadpisane w klasach bazowych

import abc

class A:
    __metaclass__ = abc.ABCMeta  # do klas abstrakcyjnych musi być dodana ABCMeta
    
    @abc.abstractmethod
    def f(self):
        pass

In [ ]:
# Python 3
# metody abstrakcyjne muszą być nadpisane w klasach bazowych

import abc

class A(metaclass=abc.ABCMeta):  # do klas abstrakcyjnych musi być dodana ABCMeta
    pass

    @abc.abstractmethod
    def f(self):
        pass

In [ ]:
# Python 2/3

# a = A()  # error

class B(A):
    pass

# b = B()  # error


class C(B):
    def f(self):
        pass
    
c = C()  # działa

Metody abstrakcyjne w Pythonie mogą mieć implementację - i można ją wywołąć używając super:

In [ ]:
# Python 2
import abc

class Base:
    __metaclass__=abc.ABCMeta
    @abc.abstractmethod
    def f(self):
        print('domyślna implementacja')
        
class A(Base):
    def f(self):
        print('A.f')
        super(A, self).f()
        
a = A()
a.f()

In [ ]:
# Python 3
import abc

class Base(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def f(self):
        print('domyślna implementacja')
        
class A(Base):
    def f(self):
        print('A.f')
        super().f()
        
a = A()
a.f()

In [ ]:
# Python 2
import abc

class A:
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractproperty
    def f(self):
        pass

In [ ]:
# Python 3
import abc

class A(metaclass=abc.ABCMeta):
    pass
    
    @abc.abstractproperty
    def f(self):
        pass

In [ ]:
# Python 2/3
    
# a = A()  # error

class B(A):
    pass

# b = B()  # error


class C(B):
    @property
    def f(self):
        pass
    
c = C()  # działa

## Trochę więcej o metodach

In [ ]:
# Python 2

class A(object):
    def f(self):
        return self
    
a = A()

print(type(A.f))  # typ to instancemethod
print(A.f)  # <unbound method A.f>
print(dir(A.f))

print()
print()
print(type(a.f))  # typ to instancemethod
print(a.f)  # <bound method A.f of <__main__.A object at ...>>
print(dir(a.f))  # metoda ma ciekawe atrybuty: __func__ i __self__

# __func__ to jest funkcja, która potem zostanie wywołana jako metoda 

print('a.f is A.f:', a.f is A.f)  # metoda a.f to coś innego niż A.f

# __self__ to jest instancja (a), która będzie przekazana do funkcji a.f.__func__ jako pierwszy argument
print('a.f.__self__ is a:', a.f.__self__ is a)
print('A.f.__self__:', A.f.__self__)  # None - nie ma __self__

# W Pythonie 2 metoda jest deskryptorem! - i to non-data deskryptorem - można przysłaniać w instancjach -
# staticmethos i classmethod też

In [ ]:
# Python 2

A.f()  # TypeError: unbound method f() must be called with A instance as first argument (got nothing instead)

In [ ]:
# Python 2
A.f(a)  # To teraz dostarczamy pierwszy argument, będący instancją klasy A

In [ ]:
# Python 2
class B(A):
    pass

b = B()
A.f(b)  # też zadziała - b jest też instancją klasy A - przez dziedziczenie

In [ ]:
# Python 2
# Metody klasy i statyczne

class A(object):
    @classmethod
    def f(cls):
        return cls
    
    @staticmethod
    def g():
        pass
    
a = A()
print(type(A.f))  # typ to 'instancemethod'
print(A.f)  # <bound method type.f of <class '__main__.A'>> - dla klasycznych metod to było unbound method
print('A.f.__self__ is A:', A.f.__self__ is A)  # __self__ jest teraz klasą - w zwykłych metodach to była instancja
                                                # albo None

print()
print()

print(type(a.f))
print(a.f)
print(dir(a.f))
print('a.f.__self__ is A:', a.f.__self__ is A)  # a.f.__self__ teraz jest klasą

In [ ]:
# Python 2
print(type(A.g))  # typ to 'function'
print(dir(A.g))  # brak __self__ - przecież jest to metoda statyczna


print(type(a.g))  # jak wcześniej
print(dir(a.g))  # jak wcześniej


In [ ]:
# Python 2
# Przysłanianie metod:

class A(object):
    def f(self):
        return 23
    
    @classmethod
    def g(cls):
        return 24
    
    @staticmethod
    def h():
        return 25
    
a = A()
print(a.f())
print(a.g())
print(a.h())

a.f = 1
a.g = 2
a.h = 3

print(a.f)
print(a.g)
print(a.h)

In [ ]:
# Python 3

class A:
    def f(self):
        return self
    
a = A()    

print(type(A.f))  # Metoda z poziomu klasy to zwykła funkcja zdefiniowana w zakresie klasy A
print(A.f)  # <function A.f at 0x102317950>
print(dir(A.f))

print()
print()

print(type(a.f))  # Metoda z poziomu instancji jest typu *method*
print(a.f)  # <bound method A.f of <__main__.A object at ...>>
print(dir(a.f))  # metoda ma ciekawe atrybuty: __func__ i __self__

# __func__ to jest funkcja, która potem zostanie wywołana jako metoda 

print('a.f is A.f:', a.f is A.f)  # metoda a.f to coś innego niż A.f
print('a.f.__func__ is A.f:', a.f.__func__ is A.f)  # ale a.f.__func__ to to samo co A.f

# __self__ to jest instancja (a), która będzie przekazana do funkcji a.f.__func__ jako pierwszy argument
print('a.f.__self__ is a.f():', a.f.__self__ is a)

In [ ]:
# Python 3

# wywołanie A.f (metody instancji) z poziomu klasy:
A.f()  # TypeError: f() missing 1 required positional argument: 'self'

In [ ]:
# Python 3
# to przekażmy parametr:
A.f(2)  # wywołaliśmy metodę klasy, jak zwykłą funkcję z argumentem nie będącym instancją klasy A -
        # ale przecież A.f to zwykła funkcja, to nie ma się co dziwić

In [ ]:
# Python 3
# Metody klasy i statyczne

class A:
    @classmethod
    def f(cls):
        return cls
    
    @staticmethod
    def g():
        pass
    
a = A()
print(type(A.f))  # typ to 'method' a nie function jak w przypadku 'klasycznych' metod
print(dir(A.f))  # jak metoda jest metodą klasy to __self__ jest już widoczne z poziomu klasy
print('A.f.__self__ is A:', A.f.__self__ is A)

print(type(a.f))
print(dir(a.f))
print('a.f.__self__ is A:', a.f.__self__ is A)  # a.f.__self__ teraz jest klasą

In [ ]:
# Python 3
print(type(A.g))  # typ to 'function'
print(dir(A.g))  # brak __self__ - przecież jest to metoda statyczna


print(type(a.g))  # jak wcześniej
print(dir(a.g)) 


In [ ]:
# Python 3
# Tak jak w Pythonie 2 można przysłaniać metody:

class A:
    def f(self):
        return 23
    
    @classmethod
    def g(cls):
        return 24
    
    @staticmethod
    def h():
        return 25
    
a = A()
print(a.f())
print(a.g())
print(a.h())

a.f = 1
a.g = 2
a.h = 3

print(a.f)
print(a.g)
print(a.h)

## Inne typy

Poza klasami tworzonymi przez słowo class i wywołanie type, do dyspozycji jest jeszcze trochę innych typów.
Można je znaleźć w module types

In [ ]:
import types

print(dir(types))

I na tej podstawie można robić np. moduły inaczej niż przez pliki:

In [ ]:
class Modul(types.ModuleType):
    x = 23
    
mod = Modul()  # TypeError: Required argument 'name' (pos 1) not found

In [ ]:
mod = Modul('mod')
print(mod.__name__)

# albo
import kot  # nie ma takiego modułu

In [ ]:
import sys
sys.modules['kot'] = Modul('kotek')  # wrzucamy moduł do listy modułów

import kot  # importuje się - moduł wstrzyknięty
print(kot.__name__)
print(kot.x)

### Podmiana typów

Typy obiektów również można zmieniać "w locie" podmieniając atrybut \_\_class\_\_ instancji:

In [ ]:
class A(object):
    x = 123  # atrybut klasy
    def __init__(self):
        self.y = 33  # ustawianie nowego atrybutu w instancji
        print("A.__init__")
        
    def f(self):
        print("A.f")
        
class B(object):
    x = 777 
    def __init__(self):
        self.y = 44  
        print("A.__init__")
        
    def f(self):
        print("B.f")    
        
a = A()
print(a.__class__)
print(a.x)
print(a.y)
a.f()

print('>>>podmiana<<<')
a.__class__ = B
print(a.__class__)  # klasa podmienione
print(a.x)  # x podmienione - bo to atrybut klasy
print(a.y)  # atrybut instancji nie zmieniony - byłby podmieniony dopiero przy wywołaniu __init__
a.f()  # metoda wywołana z klasay B


# Wniosek
# Po podmianie klasy, elementy wyciągane z klasy - czyli metody i atrybuty klasy są uaktualnione, natomiast 
# nie są uaktualnione atrybuty instancji - nie zosatło wywołane nawet __init__ (w sumie logiczne -
# podmieniony został atrybut __class__ instancji a zadna metoda nie została wywołana).
# żeby zaktualizować atrybut y trzeba ręcznie wywołać __init__

In [ ]:
a.__init__()  # podmiana atrybutu instancji
print(a.y)  # teraz nowa wartosć

## Weakref

Słabe referencje to obiekty, które nawiązują do innego obiektu, nie nie zwiększają liczby dowiązać (oryginalny obiekt nie ma zwiększonej liczby referencji)

In [ ]:
import weakref

class A(object):
    def __init__(self):
        print('{}.__init__'.format(self))
        
    def __del__(self):
        print('{}.__del__'.format(self))
        del self

a = A()

def callback(obj):
    print('callback: ', obj)

aref = weakref.ref(a, callback)
print(aref, '; ', aref())  # żeby dostać się do referenta, trzeba wywołać referencję

del a
print(aref())  # referent został usunięty, więc referencja pokazuje na None

In [ ]:
# WeakKeyDictionary - klucze są slabymi referencjami - przy tworzeniu zbioru klucze będą zamienione na
# słabe referencje

keys_list1 = [A() for i in range(5)]
d1 = weakref.WeakKeyDictionary(dict(zip(keys_list1, range(5))))
print(d1, list(d1.keys()))
print(len(d1))
keys_list1.pop()
print(len(d1))

In [ ]:
# WeakValueDictionary - podobnie jak wcześniej tylko wartosci są słabymi referencjami

keys_list2 = [A() for i in range(5)]
d2 = weakref.WeakValueDictionary(dict(enumerate(keys_list2)))
print(d2, list(d2.values()))
print(len(d2))
keys_list2.pop()
print(len(d2))

In [ ]:
# dla porównania normalny słownik
keys_list3 = [A() for i in range(5)]
d3 = dict(enumerate(keys_list3))
print(d3, list(d3.values()))
print(len(d3))
keys_list3.pop()  # nie wywołuje się __del__
print(len(d3))  # dalej jest 5 - bo w słowniku zostały utworzone nowe "twarde" 
               # referencje do obiektów i obiekt nie jest nawet usunięty

In [ ]:
# alternatywnie do słownika można uzyc zbioru
item_list = [A() for i in range(5)]
s = weakref.WeakSet(item_list)
print(s)
print(len(s))
item_list.pop()
print(len(s))

# dodatkowe __del__e w outpucie to usunięte zmienne z poprzedniego uruchomienia